In [ ]:
!pip install transformers
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel, BertConfig
from transformers import AutoModel, BertTokenizerFast
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import ndcg_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.0 MB/s eta 0:00:00


In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [ ]:
bert = BertModel.from_pretrained("bert-base-uncased", num_labels = 5, output_attentions = False, output_hidden_states = False).to(DEVICE)

# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train = pd.read_json('/content/drive/MyDrive/CUP_IT/ranking_train.jsonl', lines=True)

In [ ]:
train = train.explode(column='comments')
train['comment'] = train.comments.map(lambda dic: dic['text'])
train['score'] = train.comments.map(lambda dic: dic['score'])
train.drop(labels=['comments'], axis=1, inplace=True)
train['full_text'] = train.text + ' |||| ' + train.comment
train.head()

,text,comment,score,full_text
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0,How many summer Y Combinator fundees decided n...
0,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,1,How many summer Y Combinator fundees decided n...
0,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2,How many summer Y Combinator fundees decided n...
0,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3,How many summer Y Combinator fundees decided n...
0,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4,How many summer Y Combinator fundees decided n...


In [ ]:
features = torch.load('/content/drive/MyDrive/CUP_IT/NEW_BERT_train_tensors/100000.pt')
target = torch.tensor(train['score'].iloc[:50000].astype(np.int32).values).type(torch.LongTensor)

In [ ]:
b = torch.Tensor(len(features), 1, 128)
torch.cat(features, out=b).to(torch.int64).to(DEVICE)
train = torch.utils.data.TensorDataset(b, target)

<ipython-input-18-e80365cb77d8>:2: UserWarning: An output with one or more elements was resized since it had shape [50000, 1, 128], which does not match the required output shape [50000, 128]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:17.)
  torch.cat(features, out=b).to(torch.int64).to(DEVICE)


In [ ]:
# выделим из тренировочного датасета 20% данных для валидации
valid_ratio = 0.2
num_valid =  int(valid_ratio * len(train))
num_train = len(train) - num_valid

train_dataset, valid_dataset = random_split(dataset=train, lengths=[num_train, num_valid])

In [ ]:
batch_size = 100
train_loader = DataLoader(train_dataset, batch_size=batch_size)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

In [ ]:
def train_func(model, loader, loss_function, optimizer, scheduler):
    model.train()

    epoch_loss = 0

    for inputs in loader:
        inputs, targets = inputs[0].to(torch.int64).to(DEVICE), inputs[1].to(DEVICE)
        outputs = model(inputs)

        loss = loss_function(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    scheduler.step(epoch_loss)
    return epoch_loss / len(loader)

In [ ]:
def test(model, loader, loss_function):
    with torch.no_grad():
        model.eval()
        N = 0
        total_loss = 0.0
        correct = 0.0
        for i, inputs in enumerate(loader):
            inputs, targets = inputs[0].to(torch.int64).to(DEVICE), inputs[1].to(DEVICE)
            outputs = model(inputs)
            N += inputs.shape[0]
            loss = loss_function(outputs, targets)
            total_loss += inputs.shape[0] * loss.item()
            predicted_targets = outputs.argmax(dim=1)
            correct += (predicted_targets == targets).sum().item()
        return total_loss / N, correct / N

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      super(BERT_Arch, self).__init__()
      self.bert = bert      
      # dropout layer
      self.dropout = nn.Dropout(0.1)     
      # relu activation function
      self.relu =  nn.ReLU()
      # dense layer 1
      self.fc1 = nn.Linear(768,512)     
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,5)
      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id):
      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, return_dict=False)     
      x = self.fc1(cls_hs)
      x = self.relu(x)
      x = self.dropout(x)
      # output layer
      x = self.fc2(x)      
      # apply softmax activation
      x = self.softmax(x)
      return x

In [ ]:
#model = BertModel.from_pretrained("bert-base-uncased", num_labels = 5, output_attentions = False, output_hidden_states = False).to(DEVICE)

model = BERT_Arch(bert).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.001, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

In [ ]:
num_epochs = 10
train_losses = []
val_losses = []
for epoch in range(num_epochs):
    train_loss = train_func(model, loader=train_loader, loss_function=criterion, optimizer=optimizer, scheduler=scheduler)
    val_loss, val_acc = test(model=model, loader=valid_loader, loss_function=criterion)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    print(f'Epoch {epoch}')
    print(f'Train Loss {train_loss:.6f} Val Loss {val_loss:.6f} Val Accuracy {val_acc:.6f}')
    print('---')

#test_loss, test_acc = test(model=model, loader=test_loader, loss_function=criterion)
#print(f'Test Loss {test_loss:.6f} Test Accuracy {test_acc:.6f}')

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/CUP_IT/bert_model.pt')

In [ ]:
for i in train_loader:
  outputs = model(i[0].to(torch.int64).to(DEVICE))
        # the class with the highest energy is what we choose as prediction
  _, predicted = torch.max(outputs.data, 1)
  print(i[1])
  print(round(ndcg_score([np.asarray(i[1].cpu())], [np.asarray(predicted.cpu())]), 2))
  break

tensor([3, 0, 0, 4, 0, 3, 4, 1, 0, 1, 0, 4, 1, 0, 2, 4, 3, 3, 0, 2, 2, 1, 1, 3,
        1, 4, 3, 3, 4, 3, 1, 2, 3, 0, 3, 1, 3, 3, 0, 3, 4, 3, 2, 4, 2, 2, 3, 1,
        1, 2, 0, 0, 2, 3, 0, 0, 0, 4, 4, 0, 2, 4, 3, 3, 3, 3, 0, 0, 0, 4, 0, 2,
        0, 3, 0, 3, 0, 2, 4, 2, 1, 2, 2, 3, 4, 4, 2, 1, 2, 0, 2, 1, 1, 1, 4, 0,
        3, 2, 1, 4])
0.84
